# Functions

- [Item 14: Prefer Exceptions to Returning *None*](#Item-14:-Prefer-Exceptions-to-Returning-None)
- [Item 15: Know How Closures Interact with Variable Scope](#Item-15:-Know-How-Closures-Interact-with-Variable-Scope)
- [Item 16: Consider Generators Instead of Returning Lists]
- [Item 17: Be Defensive When Iterating Over Arguments](http://localhost:8888/notebooks/Functions.ipynb#Item-17:-Be-Defensive-When-Iterating-Over-Arguments)

## Item 14: Prefer Exceptions to Returning *None*

- Functions that return *None* to indicate special meaning are error prone because *None* and other values (e.g., zero, the empty string) all evaluate to *False* in conditional expressions.
- Raise exceptions to indicate special situations instead of returning *None*. Except the calling code to handle exceptions properly when they're documented.

In [ ]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None
    
x, y = 1, 0
result = divide(x, y)
if result is None:
    print('Invalid inputs')  # seems right
    
# what if the numerator is zero
x, y = 0, 5
result = divide(x, y)
if not result:
    print('bug 1')  # bug
    
# No.1 to fix this
def divide_s1(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None
    
success, result = divide_s1(x, y)
if not success:
    print('Invalid inputs')
    
# The problem is that callers can easily
# ignore the first part of the tuple
_, result = divide_s1(x, y)
if not result:
    print('bug 2')
    
# No.2 to fix this, never return None
def divide_s2(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e

x, y = 5, 2
try:
    result = divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print(f'Result is {result}')

## Item 15: Know How Closures Interact with Variable Scope


In [ ]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

- Python supports closures: functions that refer to variables from the scope in which they were defined. This is why the helper function is able to access the group argument to sort_priority.
- Functions are first-class objects in Python, meaning you can refer to them directly, assign them to variables, pass them as arguments to other functions, compare them in expressions and if statements, etc. This is how the sort method can accept a closure function as the key argument.
- Python has specific rules for comparing tuples. It first compares items in index zero, then index one, then index two, and so on. This is why the return value from the helper closure causes the sort order to have two distinct groups.

In [ ]:
def sort_priority2(numbers, group):
    found = False  # Scope: 'sort_priority2'
    def helper(x):
        if x in group:
            found = True  # Scope: 'helper' - Bad!
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority2(numbers, group)
print(f'Found: {found}')  ## incorrect
print(numbers)

When you reference a variable in an expression, the Python interpreter will traverse the scope to resolve the reference in this order:

1. The current function’s scope
2. Any enclosing scopes (like other containing functions)
3. The scope of the module that contains the code (also called the global scope)
4. The built-in scope (that contains functions like len and str)

If none of these places have a defined variable with the referenced name, then a NameError exception is raised.

Assigning a value to a variable works differently. If the variable is already defined in the current scope, then it will just take on the new value. If the variable doesn’t exist in the current scope, then Python treats the assignment as a variable definition. The scope of the newly defined variable is the function that contains the assignment.

Encountering this problem is sometimes called the scoping bug because it can be so surprising to newbies. But this is the intended result. This behavior prevents local variables in a function from polluting the containing module. Otherwise, every assignment within a function would put garbage into the global module scope. Not only would that be noise, but the interplay of the resulting global variables could cause obscure bugs.

### Getting Data Out

`nonlocal` won't traverse up to the **module-level scope** (to avoid polluting globals).


In [ ]:
def sort_priority3(numbers, group):
    found = False  # Scope: 'sort_priority2'
    def helper(x):
        nonlocal found
        if x in group:
            found = True  # Scope: 'helper' - Bad!
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority3(numbers, group)
print(f'Found: {found}')  ## incorrect
print(numbers)

In [ ]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True
# new problem: need to reset sorter.found manually

In [ ]:
# Python 2
def sort_priority(numbers, group):
    found = [False]
    def helper(x):
        if x in group:
            found[0] = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found[0]

## Item 16: Consider Generators Instead of Returning Lists

In [ ]:
# bad practice
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

address = 'One mile away'
result = index_words(address)
print(result)

In [ ]:
# generator
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

address = 'two mile away'
result = list(index_words_iter(address))
print(result)

In [ ]:
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

## Item 17: Be Defensive When Iterating Over Arguments

An iterator only produces its results a single time. If you iterate over an iterator or generator that has already raised a *StopIteration* exception, you won't get any results the second time around.


In [ ]:
words = ['one', 'twoo', 'threee']
it = (len(w) for w in words)
print(list(it))
print(list(it))  # Already exhausted

What's confusing is that you also won't get any errors when you iterate over an already exhausted iterator.

In [ ]:
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

data = [100, 200, 300]
percentages = normalize(data)
print(percentages)

l2g = lambda x: (n for n in x)
data_it = l2g(data)
percentages = normalize(data_it)
print(percentages)  # nothing, generator already exhausted

# Problem: input list may be too large
def normalize_copy(numbers):
    numbers = list(numbers)  # Copy the iterator
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result


# it works, but clumsy
def normalize_func(get_iter):
    total = sum(get_iter())
    result = []
    for value in get_iter():
        percent = 100 * value / total
        result.append(percent)
    return result

percentages = normalize_func(lambda: l2g(data))
print(percentages)


class IterContainer(object):
    def __init__(self, l):
        self.l = l
        
    def __iter__(self):
        for n in self.l:
            yield n
    
iter_container = IterContainer(data)
percentage = normalize(iter_container)
print(percentage)

print(iter(iter_container) is iter(iter_container))
print(iter(data_it) is iter(data_it))
print(iter(data) is iter(data))
print(type(iter(data)))
print(type(l2g(data)))


def normalize_defensive(numbers):
    if iter(numbers) is iter(numbers):  # An iterator - bad!
        raise TypeError('Cannot be a iterator')
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

normalize_defensive(iter(data))

The iterator protocol is how Python for loops and related expressions traverse the contents of a container type. When Python sees a statement like for x in foo it will actually call iter(foo). The iter built-in function calls the foo.*\_\_iter__* special method in turn. The *\_\_iter__* method must return an iterator object (which itself implements the *\_\_next__* special method). Then the for loop repeatedly calls the next built-in function on the iterator object until it’s exhausted (and raises a StopIteration exception).